In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import time
from nltk.tokenize import word_tokenize, sent_tokenize
from itertools import combinations, permutations
plt.style.use('ggplot')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df1 = pd.read_csv(r'../all-the-news/articles1.csv')
df2 = pd.read_csv(r'../all-the-news/articles2.csv')
df3 = pd.read_csv(r'../all-the-news/articles3.csv')
df = df1.append(df2).append(df3)
df.index = range(df.shape[0])
del df1, df2, df3
df.shape

(142570, 10)

In [23]:
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [5]:
len(df.content[0])

3

In [6]:
df.content[0]

0    WASHINGTON  —   Congressional Republicans have...
0    Patriots Day, Peter Berg’s new thriller that r...
0    The son of a Louisiana man whose father was sh...
Name: content, dtype: object

In [7]:
test_sample = 'Donald J. Trump is a piece of shit. He is a psychopathic maniac!'

In [8]:
word_tokenize(test_sample)

['Donald',
 'J.',
 'Trump',
 'is',
 'a',
 'piece',
 'of',
 'shit',
 '.',
 'He',
 'is',
 'a',
 'psychopathic',
 'maniac',
 '!']

In [9]:
sent_tokenize(test_sample)

['Donald J. Trump is a piece of shit.', 'He is a psychopathic maniac!']

In [10]:
df.publication.unique()

array(['New York Times', 'Breitbart', 'CNN', 'Business Insider',
       'Atlantic', 'Fox News', 'Talking Points Memo', 'Buzzfeed News',
       'National Review', 'New York Post', 'Guardian', 'NPR', 'Reuters',
       'Vox', 'Washington Post'], dtype=object)

# define biased lexicons and bigrams

In [6]:
bias_dict = {
    'New York Times': 'left-center',
    'Breitbart': 'extreme-right',
    'CNN': 'left',
    'Business Insider': 'left-center',
    'Atlantic': 'left-center',
    'Fox News': 'right',
    'Talking Points Memo': 'left',
    'Buzzfeed News': 'left-center',
    'National Review': 'right',
    'New York Post': 'right-center',
    'Guardian': 'left-center',
    'NPR': 'left-center',
    'Reuters': 'neutral',
    'Vox': 'left',
    'Washington Post': 'left-center'
}

In [7]:
for k,v in bias_dict.items():
    if 'left' in v:
        bias_dict[k] = 'liberal'
    elif 'right' in v:
        bias_dict[k] = 'conservative'
    else:
        bias_dict[k] = 'neutral'
for k,v in bias_dict.items():
    print(k,v)

New York Times liberal
Breitbart conservative
CNN liberal
Business Insider liberal
Atlantic liberal
Fox News conservative
Talking Points Memo liberal
Buzzfeed News liberal
National Review conservative
New York Post conservative
Guardian liberal
NPR liberal
Reuters neutral
Vox liberal
Washington Post liberal


In [8]:
lib_bigrams = [
    'thinkprogress org',
    'video thinkprogress',
    'et rally',
    'org 2008',
    'gi bill',
    'wonk room',
    'dana perino',
    'phil gramm',
    'senator mccain',
    'abu ghraib'
]

con_bigrams = [
    'exit question',
    'hat tip',
    'ed lasky',
    'hot air',
    'tony rezko',
    'ed morrissey',
    'track record',
    'confirmed dead',
    'american thinker',
    'illegal alien',
]

In [9]:
lib_lexicons = [
    'Administration',
    'Americans',
    'woman',
    'single',
    'personally',
    'lobbyists',
    'Republican',
    'union',
    'torture',
    'rich',
    'interests',
    'doing'
]
lib_lexicons = [lex.lower() for lex in lib_lexicons]

con_lexicons = [
    'illegal',
    'Obama’s',
    'corruption',
    'rich',
    'stop',
    'tax',
    'claimed',
    'human',
    'doesn’t',
    'difficult',
    'Democrats',
    'less'
]
con_lexicons = [lex.lower() for lex in con_lexicons]

# testing

In [24]:
df.title[0]

'House Republicans Fret About Winning Their Health Care Suit - The New York Times'

In [31]:
sentences = sent_tokenize(df.content[0])
sentences.append(df.title[0])
len(sentences)

27

In [35]:
test = [('k1','v1'),('k2','v2'),('k3','v3')]
d = pd.DataFrame(test)
d.head()

,0,1
0,k1,v1
1,k2,v2
2,k3,v3


In [39]:
d.columns = ['keys', 'vs']
d.head()

,keys,vs
0,k1,v1
1,k2,v2
2,k3,v3


In [40]:
for v in d.vs:
    print(v)

v1
v2
v3


In [54]:
?combinations

Init signature: combinations(self, /, *args, **kwargs)
Docstring:     
combinations(iterable, r) --> combinations object

Return successive r-length combinations of elements in the iterable.

combinations(range(4), 3) --> (0,1,2), (0,1,3), (0,2,3), (1,2,3)
Type:           type
Subclasses:     


In [60]:
s = sent_tokenize(df.content[0])
w = word_tokenize(s[0])
bigrams = []
for i in range(len(w) - 1):
    bigrams.append(' '.join((w[i], w[i+1])))
bigrams

['WASHINGTON —',
 '— Congressional',
 'Congressional Republicans',
 'Republicans have',
 'have a',
 'a new',
 'new fear',
 'fear when',
 'when it',
 'it comes',
 'comes to',
 'to their',
 'their health',
 'health care',
 'care lawsuit',
 'lawsuit against',
 'against the',
 'the Obama',
 'Obama administration',
 'administration :',
 ': They',
 'They might',
 'might win',
 'win .']

In [3]:
def extractContent(row):
    sentList = []
    label = bias_dict[row.publication]
    try:
        sentences = sent_tokenize(row.content)
    except:
        sentences = []
    sentences.append(row.title)
    for s in sentences:
        try:
            words = word_tokenize(s)
            words = [w.lower() for w in words]
            bigrams = []
            for i in range(len(words) - 1):
                bigrams.append(' '.join((words[i], words[i+1])))
        except:
            words = []
        if label == 'liberal':
            lexPresent = any([lex in words for lex in lib_lexicons])
            bigramPresent = any([b in bigrams for b in lib_bigrams])
        elif label == 'conservative':
            lexPresent = any([lex in words for lex in con_lexicons])
            bigramPresent = any([b in bigrams for b in con_bigrams])
        else:
            lexPresent, bigramPresent = True, True
        if lexPresent or bigramPresent:
            sentList.append((label, s))
    return sentList

In [133]:
sample = df.loc[df.publication == 'Breitbart'].iloc[10]
sample

Unnamed: 0                                                  7813
id                                                         26549
title          New Orleans Removes Famous ‘Lee Circle’ Monume...
publication                                            Breitbart
author                                               John Binder
date                                                  2017-05-20
year                                                        2017
month                                                          5
url                                                          NaN
content        NEW ORLEANS, Louisiana  —   The monument to Ge...
Name: 7813, dtype: object

In [139]:
sentList = []
sentList = extractContent(df.iloc[0])
len(sentList), len(sent_tokenize(df.content[0]))

(13, 26)

In [135]:
for i,s in enumerate(sentList):
    print(i,s)

0 ('conservative', '“The cultural and economic and the spiritual loss to this city for having those statues up that have run people out of the city,” Landrieu claimed.')


In [136]:
for i,s in enumerate(sent_tokenize(sample.content)):
    print(i,s)

0 NEW ORLEANS, Louisiana  —   The monument to General Robert E. Lee was removed from its pedestal in Lee Circle at the heart of the city of New Orleans.
1 Mayor Mitch Landrieu used the event to claim victory for political correctness.
2 [The famous Civil   monument of Robert E. Lee has been at the center of the city’s most famous traffic circle for more than a century.
3 The statue was put on the chopping block my Landrieu after he began a crusade against four monuments he deemed “symbols of white supremacy.
4 ”  Other monuments removed by the Landrieu Administration include the Battle at Liberty Place, Jefferson Davis, and the P. G. T. Beauregard Monuments, Breitbart Texas reported.
5 All the monuments were removed at some expense to taxpayers, as New Orleans Police Department (NOPD) and New Orleans Fire Department (NOFD) officials were present at each of the removal processes.
6 The money to physically remove the monuments, though, came from a private donor which Landrieu has refused

In [145]:
sents = []
s = extractContent(df.iloc[0])
len(s)

13

In [146]:
print(len(sents))
sents.extend(s)
print(len(sents))

0
13


In [149]:
sents = []
for i in range(20):
    row = df.iloc[i]
    sentList = extractContent(row)
    sents.extend(sentList)
df2 = pd.DataFrame(sents)
df2.columns = ['label', 'sentence']
df2.shape

(44, 2)

In [150]:
df2.head()

,label,sentence
0,liberal,WASHINGTON — Congressional Republicans have...
1,liberal,The incoming Trump administration could choose...
2,liberal,"In another twist, Donald J. Trump’s administra..."
3,liberal,“Given that this pending litigation involves t...
4,liberal,"“Upon taking office, the Trump administration ..."


In [4]:
def extractSentences(df):
    output = []
    for i in range(df.shape[0]):
        row = df.iloc[i]
        sentList = extractContent(row)
        output.extend(sentList)
    df = pd.DataFrame(output)
    df.columns = ['label', 'text']
    return df

In [10]:
start = time.time()
df = extractSentences(df)
print('runtime (min): ', (time.time() - start) / 60)
df.sample(5)

runtime (min):  21.079082457224526


,label,text
276171,neutral,”It’s a gut check about the health of the cons...
101859,conservative,“To me it’s [about] illegal immigration.
193337,neutral,Jay Y. Lee was named as a suspect on Wednesday...
407326,neutral,Its shares rose 2.
381828,neutral,Obama praised Merkel for her ”steady leadershi...


In [11]:
df.shape

(546544, 2)

In [16]:
df.groupby('label').count()

,text
label,
conservative,59668
liberal,181248
neutral,305627


In [12]:
!pwd

/home/alex/Documents/MIDS/w266/final_project/w266_final_project


In [17]:
df.to_csv('filtered_sentences.csv')

In [18]:
import pickle

In [21]:
?open

Signature:
open(
    file,
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
    closefd=True,
    opener=None,
)
Docstring:
Open file and return a stream.  Raise OSError upon failure.

file is either a text or byte string giving the name (and the path
if the file isn't in the current working directory) of the file to
be opened or an integer file descriptor of the file to be
wrapped. (If a file descriptor is given, it is closed when the
returned I/O object is closed, unless closefd is set to False.)

mode is an optional string that specifies the mode in which the file
is opened. It defaults to 'r' which means open for reading in text
mode.  Other common values are 'w' for writing (truncating the file if
it already exists), 'x' for creating and writing to a new file, and
'a' for appending (which on some Unix systems, means that all writes
append to the end of the file regardless of the current seek position).
In text mode, if encoding is not specified

In [26]:
start = time.time()
with open('filtered_sentences.pickle', 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)
with open('filtered_sentences.pickle', 'rb') as f:
    df2 = pickle.load(f)
time.time() - start

0.7255961894989014

In [27]:
start = time.time()
df.to_csv('filtered_sentences.csv')
df3 = pd.read_csv('filtered_sentences.csv')
time.time() - start

4.2353599071502686

In [24]:
with open('filtered_sentences.pickle', 'rb') as f:
    df2 = pickle.load(f)

In [25]:
df2.head()

,label,text
0,liberal,WASHINGTON — Congressional Republicans have...
1,liberal,The incoming Trump administration could choose...
2,liberal,"In another twist, Donald J. Trump’s administra..."
3,liberal,“Given that this pending litigation involves t...
4,liberal,"“Upon taking office, the Trump administration ..."
